In [ ]:
import pandas as pd
import pyodbc as db
import numpy as np
import os
import glob
from operator import itemgetter
from persiantools import characters
from persiantools.jdatetime import JalaliDate
import datetime
import re
from IPython.display import HTML
import itertools
from statistics import mode
from datetime import date

Connection Properties

In [ ]:
server = 'POOLBANK-RDS2'
database= 'CapitalMarket'
username= 'ah.roodsaz'
password= 'Frank137596Roodsaz'

In [ ]:
cnxn= db.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID'+username+';PWD='+password)
cursor= cnxn.cursor()

Functions

In [ ]:
def excel_to_csv(path,dirr):
    extension='xlsx'
    os.chdir(path)
    for file in os.listdir(path):
        file_xlsx= pd.read_excel(file)
        file_xlsx.to_csv(dirr+'\\'+file.split('.')[0]+'.csv',index=False)

In [ ]:
def csv_to_excel(path,dirr):
    for file in os.listdir(path):
        df=pd.read_csv(path+'\\'+file)
        df.to_csv(dirr+'\\'+file.split('.')[0]+'.xlsx',index=False)

In [ ]:
def read_merg (path,dirr,formatfile):
    # merge file to csv
    all_df=[]
    input_1=input('file name is: ')
    if formatfile=='csv':
        for file in os.listdir(path):
            df=pd.read_csv(path+'\\'+file,engine='python',encoding='utf-8-sig')
            all_df.append(df)
    if formatfile=='xlsx':
        for file in os.listdir(path):
            df=pd.read_excel(path+'\\'+file)
            all_df.append(df)
    df_raw=pd.concat(all_df,ignore_index=True).reset_index(drop=True)
    df_raw.to_csv(dirr+'\\'+f'{input_1}'+'.csv')

In [ ]:
def change_column_name (df):
    columns= df.columns.tolist()
    print(f'main column name is: {columns}')
    for i in columns:
        input_1= input(f'insert new name for {i}: ')
        df.rename(columns={i:input_1},inplace=True)

In [ ]:
def insert_update_date (df):
    input_2= input('do you want insert date column: (N\Y)')
    if input_2=='Y':
        df['insert_date']= date.today()
    input_1=input('do you want insert update date: (N\Y)')
    if input_1=='Y':
        df['update_date']= date.today()

In [ ]:
def column_clustering(df):
    columns= df.columns.tolist()
    print(f'there are headers of dataframe: {columns}')
    string_columns=[]
    code_columns=[]
    price_columns=[]
    input_1=None
    while input_1 != 'finish':
        input_1= input('choose one clustering column: (string_column,price_column,code_column)')
        if input_1 =='string_column':
            print('you choose string_column')
            input_2=None
            while input_2 !='finish':
                input_2 = input('select columns that you want: ')
                string_columns.append(input_2)
        if input_1 =='code_column':
            print('you choose code_column')
            input_3=None
            while input_3 !='finish':
                input_3 = input('select columns that you want: ')
                code_columns.append(input_3)
            code_columns.remove('finish')
        if input_1 =='price_column':
            print('you choose price_column')
            input_3=None
            while input_3 !='finish':
                input_3 = input('select columns that you want: ')
                price_columns.append(input_3)
                price_columns.remove('finish')
    string_columns.remove('finish')

    return string_columns, code_columns, price_columns

Reading data

In [ ]:
path="C:/Users/ah.roudsaz/Desktop/FullDate_correct.xlsx"

In [ ]:
df_profit= pd.read_excel(path)

In [ ]:
df_profit['MiladiModify']=df_profit['MiladiModify'].astype(str).apply(lambda x : x.replace('-',''))
df_profit['Jalali_3']=df_profit['Jalali_1'].astype(str).apply(lambda x : x.replace('/',''))

In [ ]:
df_profit.to_excel(r'C:/Users/ah.roudsaz/Desktop/FullDate_correct.xlsx')

In [ ]:
df_df_profit_c=df_profit.copy()

In [ ]:
df_profit

Creat table

In [ ]:
cursor.execute('''
		CREATE TABLE FullDate (
			Miladi date,
			Jalali_1 nvarchar(max),
			Jalali_2 nvarchar(max),
			Miladi_1 date,
			MiladiModify nvarchar(max),
			jyear nvarchar(max),
			mmonthN nvarchar(max),
			jmonthN nvarchar(max),
			mmonthT nvarchar(max),
			jmonthT nvarchar(max),
			mnime nvarchar(max),
			jnime nvarchar(max),
			JquarterN nvarchar(max),
			JQuarterT nvarchar(max),
			MquarterN nvarchar(max),
			JWeekDay nvarchar(max),
			MWeekDay nvarchar(max),
			MWeekNum nvarchar(max),
			JWeekNum nvarchar(max),
			Jalali_3 nvarchar(max),
			)
               ''')
cursor.commit()

Insert Data

In [ ]:
for index, row in df_df_profit_c.iterrows():
    cursor.execute('''
                INSERT INTO profit (description, value, date,audit_status,is_total_value,national_code,nemad,company_name,reported_period,reported_date_j,reported_date_m,reported_audit_status,is_corrected,send_time,publish_time,table_title,file_name)
                VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
                ''',
                row.loc['description'], 
                row.loc['value'],
                row.loc['date'],
                row.loc['audit_status'],
                row.loc['is_total_value'],
                row.loc['national_code'],
                row.loc['nemad'],
                row.loc['company_name'],
                row.loc['reported_period'],
                row.loc['reported_date_j'],
                row.loc['reported_date_m'],
                row.loc['reported_audit_status'],
                row.loc['is_corrected'],
                row.loc['send_time'],
                row.loc['publish_time'],
                row.loc['table_title'],
                row.loc['file_name']
                )
cnxn.commit()
cursor.close()
    

In [ ]:
pd.read_csv(r'D:\TSETMC_merge\balance_correct.csv').to_csv(r'C:\Users\ah.roudsaz\Desktop\New folder\balance_correct.csv',index=False,encoding='utf-8-sig',sep=';')